In [33]:
import pandas as pd
import os
import glob
import snowflake.connector
from IPython.display import display, Markdown

Read all files

In [ ]:

# Define the path to your data
data_path = "./"

# Load all .csv.gz files in the 'hosp' folder
hosp_files = glob.glob(os.path.join(data_path, 'hosp/*.csv.gz'))

# Create a dictionary of DataFrames
hosp_dfs = {os.path.basename(f): pd.read_csv(f) for f in hosp_files}
# # Now you can access each DataFrame by the filename, example: print(hosp_dfs['admissions.csv.gz'].head())


# Load all .csv.gz files in the 'hosp' folder
icu_files = glob.glob(os.path.join(data_path, 'icu/*.csv.gz'))

# Create a dictionary of DataFrames
icu_dfs = {os.path.basename(f): pd.read_csv(f) for f in icu_files}
# # Now you can access each DataFrame by the filename, example: print(icu_dfs['icustays.csv.gz'].head())

**Proposed Solution**


MIMIC-IV Data pre-processingL generating text from all data associated with a patient.

In [35]:

def generate_summary_from_csv(patient_id, df, key_column='subject_id'):
    """
    Generates a summary for a given patient based on the information available in a single DataFrame.
    
    Parameters:
    - patient_id: The ID of the patient.
    - df: The DataFrame containing data for the patient.
    - key_column: The column to use as a key for filtering the DataFrame (default is 'subject_id').
    
    Returns:
    - A summary string.
    """
    # Check if the key_column exists in the DataFrame
    if key_column not in df.columns:
        return None
    
    # Filter the DataFrame for the specific patient
    patient_data = df[df[key_column] == patient_id]
    
    if patient_data.empty:
        return None
    
    row = patient_data.iloc[0]  # Take the first row for simplicity
    
    summary_parts = []
    for column_name, value in row.items():
        if pd.notna(value):
            summary_parts.append(f"{column_name}: {value}")
    
    return ". ".join(summary_parts) + "."

def generate_aggregated_summary(patient_id, hosp_dfs, icu_dfs):
    summary_parts = []
    
    # Generate summaries from hospital DataFrames
    for file_name, df in hosp_dfs.items():
        key_column = 'subject_id' if 'subject_id' in df.columns else 'hadm_id'
        summary = generate_summary_from_csv(patient_id, df, key_column=key_column)
        if summary:
            summary_parts.append(summary)
    
    # Generate summaries from ICU DataFrames
    for file_name, df in icu_dfs.items():
        key_column = 'subject_id' if 'subject_id' in df.columns else 'stay_id'
        summary = generate_summary_from_csv(patient_id, df, key_column=key_column)
        if summary:
            summary_parts.append(summary)
    
    return " ".join(summary_parts)

# Get a list of all unique patient IDs (from any DataFrame, such as patients.csv.gz)
patient_ids = hosp_dfs['patients.csv.gz']['subject_id'].unique()

# Dictionary to store the summaries
all_patient_summaries = {}

# Generate summaries for each patient
for patient_id in patient_ids:
    summary = generate_aggregated_summary(patient_id, hosp_dfs, icu_dfs)
    all_patient_summaries[patient_id] = summary
    # break

# Convert to DataFrame for easy handling
summary_df = pd.DataFrame(list(all_patient_summaries.items()), columns=['subject_id', 'patient_summary'])
# print(summary_df['subject_id'].values[0], ' : ', summary_df['patient_summary'].values[0])
summary_df.to_csv('to_sf.csv')


**Snowflake Connection** 

I have credentials listed here in case you want to go into this test snowflake instance and explore it.

In [36]:
conn = snowflake.connector.connect(
    account = "ibb98831",
    user = "puneetjain",
    password = "tBowz$M136o*YG1d",
    role = "ACCOUNTADMIN",
    warehouse = "COMPUTE_WH",
    database="PATIENT_TEXTS",
    schema="PUBLIC",
    region="us-west-2"
    )

cur = conn.cursor()

# I have already put the above summary on snowflake, so just going to query it
cur.execute("SELECT * FROM PATIENT_TEXTS.PUBLIC.PATIENTSUMMARY LIMIT 1")

row = cur.fetchone()
print(row)


(0, '10014729', 'poe_id: 10014729-590. poe_seq: 590. subject_id: 10014729. hadm_id: 23300884. ordertime: 2125-03-26 09:02:16. order_type: Medications. transaction_type: New. order_provider_id: P00AEX. order_status: Inactive. poe_id: 10014729-518. poe_seq: 518. subject_id: 10014729. field_name: Admit to. field_value: Surgery. subject_id: 10014729. gender: F. anchor_age: 21. anchor_year: 2125. anchor_year_group: 2011 - 2013. subject_id: 10014729. hadm_id: 23300884. seq_num: 3. icd_code: 99832. icd_version: 9. subject_id: 10014729. hadm_id: 23300884. pharmacy_id: 8661097. poe_id: 10014729-356. poe_seq: 356.0. order_provider_id: P98O2Z. starttime: 2125-03-19 20:00:00. stoptime: 2125-03-20 18:00:00. drug_type: MAIN. drug: Insulin. formulary_drug_cd: INRHI. gsn: 001723. ndc: 2821501.0. prod_strength: 100 Units / mL - 10 mL Vial. dose_val_rx: 0. dose_unit_rx: UNIT. form_val_disp: 0. form_unit_disp: mL. route: SC. subject_id: 10014729. hadm_id: 28889419. drg_type: HCFA. drg_code: 908. descript

Now let's look at the questions we ask, and their answers.
---

**What is the medication history of the patient from the following**

In [37]:

query = """select SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.1-405b', CONCAT('What is the medication history of the patient from the following: ', C3)) FROM PATIENT_TEXTS.PUBLIC.PATIENTSUMMARY LIMIT 1;"""

cur.execute(query)

row = cur.fetchone()
display(Markdown(row[0]))

Based on the provided data, the patient's medication history includes:

1. **Insulin**:
	* Order date: 2125-03-19 20:00:00
	* Stop date: 2125-03-20 18:00:00
	* Dose: 0 units ( unclear, possibly an error)
	* Route: Subcutaneous (SC)
2. **Chlorhexidine Gluconate 0.12% Oral Rinse**:
	* Order date: 2125-02-27 16:30:43
	* Start date: 2125-02-27 20:00:00
	* Stop date: 2125-02-27 17:00:00 ( unclear, possibly an error)
	* Frequency: Twice a day (BID)
	* Route: Oral
	* Status: Discontinued

Additionally, there are mentions of other medications and treatments, such as:

* A medication with an order ID of 6089990, which had a change in dose or rate.
* A fluid (crystalloid) with an order ID of 1673712, which was administered continuously.
* A ventilation treatment with an order ID of 4513647, which was a continuous process.

Please note that the data appears to be incomplete, and some information may be missing or incorrect. A thorough review of the patient's medical records would be necessary to obtain an accurate and comprehensive medication history.

**What are the top 3 risk factors for this patient based on this clinical history**

In [38]:
query = """select SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.1-405b', CONCAT('What are the top 3 risk factors for this patient based on this clinical history: ', C3)) FROM PATIENT_TEXTS.PUBLIC.PATIENTSUMMARY LIMIT 1;"""

cur.execute(query)

row = cur.fetchone()
display(Markdown(row[0]))

Based on the provided clinical history, the top 3 risk factors for this patient are:

1. **Methicillin-Resistant Staphylococcus aureus (MRSA) colonization**: The patient has a positive MRSA screen, which increases the risk of developing MRSA infections, particularly in a hospital setting.
2. **Recent surgery**: The patient was admitted to the surgery department and had a recent surgical procedure, which increases the risk of post-operative complications, such as infection, bleeding, or respiratory problems.
3. **Insulin therapy**: The patient is receiving insulin, which suggests that they have diabetes or hyperglycemia. This increases the risk of developing complications such as diabetic ketoacidosis, hypoglycemia, or wound infections, particularly in the post-operative period.

These risk factors are not exhaustive, and a comprehensive assessment of the patient's medical history, laboratory results, and current condition would be necessary to identify all potential risk factors. However, based on the provided information, these three factors appear to be significant concerns for this patient.

**What is the hospitalization history of the patient**

In [39]:
query = """select SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.1-405b', CONCAT('Give me a short summary of the hospitalization history of the patient from the following: ', C3)) FROM PATIENT_TEXTS.PUBLIC.PATIENTSUMMARY LIMIT 1;"""

cur.execute(query)

row = cur.fetchone()
display(Markdown(row[0]))

Based on the provided data, here is a short summary of the hospitalization history of the patient with subject_id 10014729:

**Admission 1 (hadm_id: 28889419)**

* Admission date: 2125-02-27
* Admission location: Not specified
* Discharge date: Not specified (but a discharge event is mentioned with eventtype: discharge)
* Length of stay (LOS): 2.47 days (in the Cardiac Vascular Intensive Care Unit (CVICU))
* Diagnosis: Not explicitly mentioned, but the patient had a Staph aureus screen with a positive result for methicillin-resistant Staph aureus (MRSA)
* Procedures/Treatments:
	+ Received Chlorhexidine Gluconate 0.12% Oral Rinse
	+ Had a continuous IV with additives
	+ Was on ventilation

**Admission 2 (hadm_id: 23300884)**

* Admission date: 2125-03-19
* Admission location: Emergency Room
* Admission type: Emergency
* Discharge date: 2125-03-28
* Discharge location: Home Health Care
* Length of stay (LOS): Not explicitly mentioned
* Diagnosis: Not explicitly mentioned, but the patient had ICD-9 codes 99832 and 3897
* Procedures/Treatments:
	+ Received Insulin
	+ Had a surgery (based on the "Admit to" field value: Surgery)
	+ Was transferred from VSURG to CSURG

Please note that this summary is based on a limited subset of data and may not be comprehensive or entirely accurate.

**What are the demographics and personal details of the patient**

In [40]:
query = """select SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.1-405b', CONCAT('What are the demographics and personal details of the patient from the following: ', C3)) FROM PATIENT_TEXTS.PUBLIC.PATIENTSUMMARY LIMIT 1;"""

cur.execute(query)

row = cur.fetchone()
display(Markdown(row[0]))

Based on the provided data, here are the demographics and personal details of the patient:

1. **Subject ID**: 10014729
2. **Gender**: Female (F)
3. **Age**: 21 (as of the anchor year 2125)
4. **Anchor Year Group**: 2011-2013
5. **Marital Status**: Single
6. **Race**: White - Other European
7. **Language**: English
8. **Height**: 65 inches (approximately 5 feet 5 inches)
9. **Weight**: 72.5 (units not specified, but likely in kilograms or pounds)

Please note that some of these details may be mentioned multiple times in the data, but I have only extracted each piece of information once. Additionally, some information, such as the patient's name, date of birth, and contact information, is not present in the provided data.

In [41]:
cur.close()
conn.close()

**Now, let's look at the other ChargeMaster data**
---

In [42]:
conn = snowflake.connector.connect(
    account = "ibb98831",
    user = "puneetjain",
    password = "tBowz$M136o*YG1d",
    role = "ACCOUNTADMIN",
    warehouse = "COMPUTE_WH",
    database="HOSPITAL_CHARGEMASTER_DATA",
    schema="ISTG",
    region="us-west-2"
    )

cur = conn.cursor()

cur.execute("select * from HOSPITAL_CHARGEMASTER_DATA.ISTG.CHARGE_MASTER limit 1;")

row = cur.fetchone()
print(row)

(49230767, 12, '30000323', 'HC COMPL CBC W/PLT W/AUTOM DIFF', '135', 1301071, '0', Decimal('1.00000'), datetime.datetime(2021, 10, 1, 0, 0), datetime.datetime(9999, 12, 31, 0, 0), None)


Let's look at some of the questions we asked from it

**What are the patients previous diagnoses and future risk factors from this chronological list of diagnoses**

In [43]:
query = """select RECORD_ID,SNOWFLAKE.CORTEX.COMPLETE('llama3.1-405b', CONCAT('What are the patients previous diagnoses and future risk factors from this chronological list of diagnoses: ', DESC)) FROM (
select RECORD_ID, LISTAGG(ICD_DIAGNOSIS_DESC, ', ')
WITHIN GROUP (ORDER BY DIAGNOSIS_ORDER  DESC) as "DESC"
  FROM HOSPITAL_CHARGEMASTER_DATA.ISTG.DIAGNOSES 
  group by RECORD_ID 
   limit 1);"""
cur.execute(query)

row = cur.fetchone()
display(Markdown(row[1]))

What a complex medical history!

From this chronological list of diagnoses, we can infer the patient's previous diagnoses and future risk factors as follows:

**Previous Diagnoses:**

1. **Diffuse large B-cell lymphoma** (twice mentioned, indicating a relapse or ongoing treatment): a type of non-Hodgkin lymphoma, a cancer of the immune system.
2. **Immunodeficiency due to drugs**: likely a result of chemotherapy or immunosuppressive therapy for lymphoma.
3. **Antineoplastic chemotherapy-induced pancytopenia**: a condition where chemotherapy has caused a decrease in blood cells, including red blood cells, white blood cells, and platelets.
4. **Syndrome of inappropriate secretion of antidiuretic hormone (SIADH)**: a condition where the body produces too much antidiuretic hormone, leading to water retention and electrolyte imbalances.
5. **Aphasia**: a condition affecting language processing, possibly related to the lymphoma or its treatment.
6. **Encephalopathy**: a condition affecting brain function, possibly related to the lymphoma, its treatment, or other factors.
7. **Unspecified severe protein-calorie malnutrition**: a condition indicating inadequate nutrition, possibly related to the lymphoma, its treatment, or other factors.
8. **Depression**: a mental health condition that may be related to the patient's medical history or other factors.
9. **Adult failure to thrive**: a condition indicating inadequate nutrition, weight loss, or other factors affecting overall health.
10. **COVID-19**: the patient has a history of COVID-19 infection.

**Future Risk Factors:**

1. **Infections**: due to immunodeficiency and pancytopenia, the patient is at increased risk of developing infections.
2. **Bleeding or thrombotic events**: due to coagulation defects and pancytopenia, the patient is at increased risk of bleeding or blood clotting disorders.
3. **Cardiovascular events**: due to hypertension, hypercholesterolemia, and atrial fibrillation, the patient is at increased risk of cardiovascular events, such as heart attacks or strokes.
4. **Malnutrition and weight loss**: due to the history of severe protein-calorie malnutrition and adult failure to thrive, the patient is at increased risk of malnutrition and weight loss.
5. **Neurological complications**: due to the history of encephalopathy, aphasia, and immune effector cell-associated neurotoxicity syndrome, the patient is at increased risk of neurological complications.
6. **Secondary cancers**: due to the history of lymphoma and radiation therapy, the patient is at increased risk of developing secondary cancers.

Please note that this is not an exhaustive list, and the patient's actual risk factors may be more extensive. A thorough medical evaluation and ongoing monitoring are essential to manage these conditions and prevent potential complications.

**Assign a risk score to the patient - with 0 being low risk and 100 being very high risk**

In [44]:
query = """select RECORD_ID,SNOWFLAKE.CORTEX.COMPLETE('llama3.1-405b', CONCAT('Assign a risk score to the patient - with 0 being low risk and 100 being very high risk: ', DESC)) FROM (
select RECORD_ID, LISTAGG(ICD_DIAGNOSIS_DESC, ', ')
WITHIN GROUP (ORDER BY DIAGNOSIS_ORDER  DESC) as "DESC"
  FROM HOSPITAL_CHARGEMASTER_DATA.ISTG.DIAGNOSES 
  group by RECORD_ID
   limit 3 );"""
cur.execute(query)

row = cur.fetchall()


In [45]:
for r in row:
    print('patient: ')
    display(Markdown(r[0]))
    print('risk: ')
    display(Markdown(r[1]))
    print('------------------------------------')
    print('------------------------------------')

patient: 


1895628278

risk: 


Based on the provided medical conditions, I would assign a risk score of 92 to this patient. Here's a breakdown of the factors that contribute to this score:

**High-risk conditions:**

1. **Diffuse large B-cell lymphoma** (twice mentioned): This is an aggressive type of cancer that significantly increases the patient's risk.
2. **Immunodeficiency due to drugs**: This condition compromises the patient's immune system, making them more susceptible to infections and other complications.
3. **Antineoplastic chemotherapy induced pancytopenia**: This condition indicates a severe decrease in blood cells, which can lead to infections, bleeding, and anemia.
4. **Cytokine release syndrome, grade 1**: Although grade 1 is considered mild, this condition can still lead to serious complications, such as organ failure.
5. **Encephalopathy, unspecified**: This condition indicates brain damage or disease, which can significantly impact the patient's quality of life and increase their risk.
6. **Unspecified severe protein-calorie malnutrition**: Malnutrition can weaken the patient's immune system and increase their risk of complications.
7. **Adult failure to thrive**: This condition indicates a decline in the patient's overall health and well-being.

**Moderate-risk conditions:**

1. **Delirium due to known physiological condition**: Delirium can increase the patient's risk of falls, injuries, and other complications.
2. **Aphasia**: This condition can lead to communication difficulties and increase the patient's risk of misunderstandings and errors.
3. **Unspecified atrial fibrillation**: This condition can increase the patient's risk of stroke and other cardiovascular complications.
4. **Essential (primary) hypertension**: Uncontrolled hypertension can lead to cardiovascular complications.
5. **Pure hypercholesterolemia, unspecified**: High cholesterol can increase the patient's risk of cardiovascular disease.

**Low-risk conditions:**

1. **Fever, unspecified**: While fever can be a concern, it is a relatively common and treatable condition.
2. **Constipation, unspecified**: This condition is typically not life-threatening and can be managed with treatment.
3. **Body mass index [BMI] 23.0-23.9, adult**: A normal BMI does not significantly contribute to the patient's risk.

**Other factors:**

1. **Personal history of irradiation**: This may increase the patient's risk of certain cancers and other complications.
2. **Personal history of nicotine dependence**: This may increase the patient's risk of respiratory and cardiovascular complications.
3. **Personal history of COVID-19**: This may increase the patient's risk of respiratory complications.

Overall, the patient's multiple high-risk conditions, including cancer, immunodeficiency, and pancytopenia, contribute to a high risk score. The presence of moderate-risk conditions, such as delirium and aphasia, further increases the patient's risk. While there are some low-risk conditions present, they do not significantly impact the overall risk score.

------------------------------------
------------------------------------
patient: 


...

------------------------------------
------------------------------------
patient: 


1602617727

risk: 


Based on the provided medical conditions, I would assign a risk score of **95** to the patient. Here's a breakdown of the factors contributing to this score:

**High-risk conditions:**

1. Severe sepsis with septic shock: This is a life-threatening condition that requires immediate attention. (Risk score: 30)
2. Secondary malignant neoplasms (multiple sites): The presence of multiple secondary cancers indicates advanced disease and a high risk of complications. (Risk score: 20)
3. Acute kidney failure: This condition can lead to serious complications, including organ failure and death. (Risk score: 15)
4. Agranulocytosis secondary to cancer chemotherapy: This condition increases the risk of severe infections. (Risk score: 10)
5. Old myocardial infarction and cardiomyopathy: These conditions indicate a history of cardiovascular disease, which increases the risk of future cardiac events. (Risk score: 5)

**Moderate-risk conditions:**

1. Pancytopenia: This condition increases the risk of infections and bleeding. (Risk score: 5)
2. Thrombocytopenia: This condition increases the risk of bleeding. (Risk score: 3)
3. Hypokalemia: This condition can lead to cardiac arrhythmias and muscle weakness. (Risk score: 2)
4. Fluid overload: This condition can lead to respiratory and cardiac complications. (Risk score: 2)
5. Opioid dependence: This condition increases the risk of overdose and other complications. (Risk score: 2)

**Low-risk conditions:**

1. Essential hypertension: This condition is a risk factor for cardiovascular disease, but it is not immediately life-threatening. (Risk score: 1)
2. Hypothyroidism: This condition can be managed with medication and is not typically life-threatening. (Risk score: 1)
3. Claustrophobia and major depressive disorder: These conditions can impact quality of life, but they are not typically life-threatening. (Risk score: 1)

**Other factors:**

1. History of cancer and chemotherapy: This increases the risk of future complications and secondary cancers. (Risk score: 5)
2. Multiple hospitalizations and non-institutional care: This indicates a high level of medical complexity and increases the risk of hospital-acquired infections and other complications. (Risk score: 5)

Overall, the patient's multiple high-risk conditions, including severe sepsis, secondary malignant neoplasms, and acute kidney failure, contribute to a high risk score. The presence of moderate-risk conditions, such as pancytopenia and thrombocytopenia, further increases the risk score. While the patient's low-risk conditions, such as essential hypertension and hypothyroidism, do not significantly impact the overall risk score, they do contribute to the patient's overall medical complexity.

------------------------------------
------------------------------------


In [46]:
cur.close()
conn.close()


In [60]:
conn = snowflake.connector.connect(
    account = "ibb98831",
    user = "puneetjain",
    password = "tBowz$M136o*YG1d",
    role = "ACCOUNTADMIN",
    warehouse = "COMPUTE_WH",
    database="HOSPITAL_CHARGEMASTER_DATA",
    schema="ISTG",
    region="us-west-2"
    )

cur = conn.cursor()
query = """select RECORD_ID,SNOWFLAKE.CORTEX.COMPLETE('llama3.1-8b', CONCAT('Assign a risk score to the patient - with 0 being low risk and 100 being very high risk: ', DESC)) FROM (
  select RECORD_ID, LISTAGG(ICD_DIAGNOSIS_DESC, ', ')
  WITHIN GROUP (ORDER BY DIAGNOSIS_ORDER  DESC) as "DESC"
  FROM HOSPITAL_CHARGEMASTER_DATA.ISTG.DIAGNOSES 
  group by RECORD_ID
  limit 1);"""
cur.execute(query)

row = cur.fetchall()
display(Markdown(row[0][1]))
cur.close()
conn.close()

To assign a risk score to the patient based on the provided list of conditions, I'll use a general framework that considers the severity and potential impact of each condition on the patient's health. This will be a subjective assessment, and actual risk scores may vary depending on the specific context and clinical judgment.

Here's a risk score assignment for each condition, with 0 being low risk and 100 being very high risk:

1. **Delirium due to known physiological condition**: 60 (moderate to high risk due to potential for acute cognitive decline and increased risk of falls, infections, and other complications)
2. **Other specified coagulation defects**: 80 (high risk due to potential for severe bleeding or thrombotic events)
3. **Immunodeficiency due to drugs**: 70 (moderate to high risk due to increased susceptibility to infections)
4. **Syndrome of inappropriate secretion of antidiuretic hormone**: 60 (moderate to high risk due to potential for hyponatremia and fluid overload)
5. **Aphasia**: 40 (moderate risk due to potential for communication difficulties and impact on daily functioning)
6. **Immune effector cell-associated neurotoxicity synd, grade 3**: 90 (very high risk due to severe neurological complications)
7. **Unspecified severe protein-calorie malnutrition**: 80 (high risk due to potential for weight loss, muscle wasting, and increased risk of infections)
8. **Unspecified fall, initial encounter**: 50 (moderate risk due to potential for injury and increased risk of future falls)
9. **Depression, unspecified**: 30 (low to moderate risk due to potential for impact on quality of life, but generally manageable with treatment)
10. **Adult failure to thrive**: 60 (moderate to high risk due to potential for malnutrition, weight loss, and increased risk of infections)
11. **Adverse effect of antineoplastic and immunosup drugs, init**: 80 (high risk due to potential for severe side effects and increased risk of infections)
12. **Other long term (current) drug therapy**: 20 (low risk due to generally manageable side effects and potential benefits)
13. **Personal history of irradiation**: 40 (moderate risk due to potential for long-term side effects, but generally manageable)
14. **Personal history of nicotine dependence**: 20 (low risk due to generally manageable condition with treatment)
15. **Antineoplastic chemotherapy induced pancytopenia**: 90 (very high risk due to severe blood cell deficiencies and increased risk of infections)
16. **Personal history of COVID-19**: 40 (moderate risk due to potential for long-term effects, but generally manageable)
17. **Encntr for exam for nrml cmprsn and ctrl in clncl rsrch prog**: 10 (low risk due to generally routine and non-invasive procedure)
18. **Cytokine release syndrome, grade 1**: 50 (moderate risk due to potential for systemic inflammation and organ dysfunction)
19. **Fever, unspecified**: 30 (low to moderate risk due to generally manageable condition with treatment)
20. **Body mass index [BMI] 23.0-23.9, adult**: 10 (low risk due to generally healthy BMI)
21. **Constipation, unspecified**: 20 (low risk due to generally manageable condition with treatment)
22. **Unspecified atrial fibrillation**: 60 (moderate to high risk due to potential for stroke and other cardiovascular complications)
23. **Essential (primary) hypertension**: 40 (moderate risk due to potential for cardiovascular complications, but generally manageable with treatment)
24. **Pure hypercholesterolemia, unspecified**: 30 (low to moderate risk due to potential for cardiovascular complications, but generally manageable with treatment)
25. **Encephalopathy, unspecified**: 80 (high risk due to potential for severe neurological complications)
26. **Diffuse large B-cell lymphoma, unspecified site**: 90 (very high risk due to potential for aggressive cancer progression and treatment-related complications)
27. **Diffuse large B-cell lymphoma, unspecified site**: 90 (very high risk due to potential for aggressive cancer progression and treatment-related complications)

To calculate a total risk score, I'll assign a weighted average based on the number of conditions and their respective risk scores. This is a simplified approach and may not reflect the actual risk assessment in a clinical setting.

**Total risk score:** 64.4 (out of 100)

This score suggests a moderate to high risk for the patient, with several conditions contributing to the overall risk. However, please note that this is a simplified assessment and actual risk scores may vary depending on the specific context and clinical judgment.

In [59]:
conn = snowflake.connector.connect(
    account = "ibb98831",
    user = "puneetjain",
    password = "tBowz$M136o*YG1d",
    role = "ACCOUNTADMIN",
    warehouse = "COMPUTE_WH",
    database="PATIENT_TEXTS",
    schema="PUBLIC",
    region="us-west-2"
    )

cur = conn.cursor()

# I have already put the above summary on snowflake, so just going to query it
cur.execute("SELECT * FROM PATIENT_TEXTS.PUBLIC.PATIENTSUMMARY LIMIT 1")

row = cur.fetchone()
print(row[2])

query = """select SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.1-8b', CONCAT('What are the top 3 risk factors for this patient based on this clinical history: ', C3)) FROM PATIENT_TEXTS.PUBLIC.PATIENTSUMMARY LIMIT 1;"""

cur.execute(query)

row = cur.fetchall()
display(Markdown(row[0][0]))
cur.close()
conn.close()

poe_id: 10014729-590. poe_seq: 590. subject_id: 10014729. hadm_id: 23300884. ordertime: 2125-03-26 09:02:16. order_type: Medications. transaction_type: New. order_provider_id: P00AEX. order_status: Inactive. poe_id: 10014729-518. poe_seq: 518. subject_id: 10014729. field_name: Admit to. field_value: Surgery. subject_id: 10014729. gender: F. anchor_age: 21. anchor_year: 2125. anchor_year_group: 2011 - 2013. subject_id: 10014729. hadm_id: 23300884. seq_num: 3. icd_code: 99832. icd_version: 9. subject_id: 10014729. hadm_id: 23300884. pharmacy_id: 8661097. poe_id: 10014729-356. poe_seq: 356.0. order_provider_id: P98O2Z. starttime: 2125-03-19 20:00:00. stoptime: 2125-03-20 18:00:00. drug_type: MAIN. drug: Insulin. formulary_drug_cd: INRHI. gsn: 001723. ndc: 2821501.0. prod_strength: 100 Units / mL - 10 mL Vial. dose_val_rx: 0. dose_unit_rx: UNIT. form_val_disp: 0. form_unit_disp: mL. route: SC. subject_id: 10014729. hadm_id: 28889419. drg_type: HCFA. drg_code: 908. description: OTHER O.R. P

Based on the provided clinical history, the top 3 risk factors for this patient are:

1. **Methicillin-resistant Staphylococcus aureus (MRSA) infection**: The patient has a positive test result for MRSA, which is a type of bacterial infection that is resistant to many antibiotics. This increases the risk of complications, prolonged hospital stay, and increased healthcare costs.
2. **Surgical procedure**: The patient underwent a surgical procedure (ICD-9 code 99832), which increases the risk of post-operative complications, such as infection, bleeding, and respiratory problems.
3. **Insulin therapy**: The patient is receiving insulin therapy (100 Units/mL - 10 mL Vial), which increases the risk of hypoglycemia (low blood sugar), hyperglycemia (high blood sugar), and other complications related to diabetes management.

These three risk factors are the most significant based on the provided clinical history, but it's essential to note that there may be other risk factors present that are not mentioned in the provided data.